In [10]:
#last fm
URL_BASE = "http://ws.audioscrobbler.com/2.0/"


# Conexiones e importaciones

In [11]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

In [ ]:
auth_manager = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIPY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")
)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
datos = sp.search(q="genre:flamenco year:2010", type='track', limit=50, offset=0)

In [ ]:
print(datos)